In [ ]:
!pip install -r requirements.txt -q


In [1]:
from rank_bm25 import BM25Okapi
import pandas as pd
import nltk.corpus
import nltk
import nltk.stem
import rich
import re
import helper
import importlib

In [2]:
importlib.reload(helper)

<module 'helper' from '/notebooks/llm_search/notebooks/helper.py'>

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
def my_awesome_tokenizer(content, min_token_length=2):
    content = content.lower()
    
    # only keep alphanumeric
    content = re.sub(r'\W+', ' ', content)
    
    
    tokens = nltk.word_tokenize(content)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    tokens_final = []
    
    for token in tokens:
        if  (   ( len(token) >= min_token_length )  and 
                ( token not in  nltk.corpus.stopwords.words("english")  )  
            ):
            
            token =lemmatizer.lemmatize(token)
            
            tokens_final.append(token)
            
            
    return tokens_final

In [5]:
def find_results(query:str ,df_corpus:pd.DataFrame, k =20):
    
    tokenized_query = my_awesome_tokenizer(query)
    doc_scores = bm25.get_scores(tokenized_query)
    
    tokens_in_vocab = [ token for token in tokenized_query  if token in bm25.idf]
    top_items = doc_scores.argsort()[-k:][::-1]
    
    degug_info = {
         "query_original":  query 
         , "tokenized_query":  tokenized_query 
         , "tokens_in_vocab" : tokens_in_vocab
        , "doc_scores":  doc_scores 
        , "top_items":  top_items 
    }
    rich.print (degug_info )
    
    df_subset = df_corpus.iloc[top_items]
    
    
    
    rich.print ( list (df_subset['product_title'] ) )
    
    helper.display_results(query,df_subset)



In [6]:
df = pd.read_parquet("../data/cleaned_input.parquet")

In [7]:
df.iloc[0].to_dict()

{'query': ' revent 80 cfm',
 'product_id': 'B000MOO21W',
 'relevance_label': 'Irrelevant',
 'product_title': 'Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceiling Mounted Fan',
 'product_description': None,
 'product_bullet_point': 'WhisperCeiling fans feature a totally enclosed condenser motor and a double-tapered, dolphin-shaped bladed blower wheel to quietly move air\nDesigned to give you continuous, trouble-free operation for many years thanks in part to its high-quality components and permanently lubricated motors which wear at a slower pace\nDetachable adaptors, firmly secured duct ends, adjustable mounting brackets (up to 26-in), fan/motor units that detach easily from the housing and uncomplicated wiring all lend themselves to user-friendly installation\nThis Panasonic fan has a built-in damper to prevent backdraft, which helps to prevent outside air from coming through the fan\n0.35 amp',
 'product_brand': 'Panasonic',
 'product_color': 'White',
 'url_product': 'https://www.amazo

In [ ]:
df_product = df[['product_id','product_title','url_product','url_image']].drop_duplicates(['product_id']).reset_index(drop=True)
df_product

In [ ]:
corpus = df_product['product_title']

In [ ]:
bm25 = BM25Okapi(corpus=corpus, tokenizer=my_awesome_tokenizer)


In [ ]:
query = "adidas shoes men 15"

In [ ]:
find_results( query, df_product)
